In [28]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import numpy as np
import requests
import hopsworks
from sklearn.impute import SimpleImputer
import os
%load_ext dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [29]:
%dotenv -vo .env

In [30]:
print(os.environ.get('EIA_API_KEY'))

url = ('https://api.eia.gov/v2/electricity/rto/daily-region-data/data/'
       '?frequency=daily'
       '&data[0]=value'
       '&facets[respondent][]=NY'
       '&facets[timezone][]=Eastern'
       '&facets[type][]=D'
       '&start=2017-01-01'
       '&end=2021-12-31'
       '&sort[0][column]=period'
       '&sort[0][direction]=desc'
       '&offset=0'
       '&length=5000'
       '&api_key=') + os.environ.get('EIA_API_KEY')

gxfTV1lJaU0R5y0MFUjNR0GurPkbLk7a8ZrIJbk1


In [31]:
data = requests.get(url).json()['response']['data']
data = pd.DataFrame(data)

In [32]:
display(data.head(5))

,period,respondent,respondent-name,type,type-name,timezone,timezone-description,value,value-units
0,2021-12-31,NY,New York,D,Demand,Eastern,Eastern,380279,megawatthours
1,2021-12-30,NY,New York,D,Demand,Eastern,Eastern,401376,megawatthours
2,2021-12-29,NY,New York,D,Demand,Eastern,Eastern,411729,megawatthours
3,2021-12-28,NY,New York,D,Demand,Eastern,Eastern,411471,megawatthours
4,2021-12-27,NY,New York,D,Demand,Eastern,Eastern,424491,megawatthours


In [33]:
demand_daily = data[['period', 'value']].rename(columns={'period': 'date', 'value': 'demand'})
demand_daily['date'] = pd.to_datetime(demand_daily['date'], infer_datetime_format=True)
display(demand_daily.head(5))

,date,demand
0,2021-12-31,380279
1,2021-12-30,401376
2,2021-12-29,411729
3,2021-12-28,411471
4,2021-12-27,424491


In [34]:
display(demand_daily.dtypes)

date      datetime64[ns]
demand             int64
dtype: object

In [35]:
# read weather data (dataset years 2017-2021 for LaGuardia from https://www.ncdc.noaa.gov/, daily summaries in climate data online)
weather_data = pd.read_csv('./noaa_weather_lag_2017-2021.csv')
display(weather_data.head(5))

,STATION,NAME,DATE,TAVG
0,USW00014732,"LAGUARDIA AIRPORT, NY US",2017-01-01,8.0
1,USW00014732,"LAGUARDIA AIRPORT, NY US",2017-01-02,4.4
2,USW00014732,"LAGUARDIA AIRPORT, NY US",2017-01-03,5.6
3,USW00014732,"LAGUARDIA AIRPORT, NY US",2017-01-04,8.2
4,USW00014732,"LAGUARDIA AIRPORT, NY US",2017-01-05,1.0


In [36]:
weather_daily = weather_data[['DATE', 'TAVG']].rename(columns={'DATE': 'date', 'TAVG': 'temperature'})
weather_daily['date'] = pd.to_datetime(weather_daily['date'], infer_datetime_format=True)
display(weather_daily.head(5))

,date,temperature
0,2017-01-01,8.0
1,2017-01-02,4.4
2,2017-01-03,5.6
3,2017-01-04,8.2
4,2017-01-05,1.0


In [37]:
display(weather_daily.dtypes)

date           datetime64[ns]
temperature           float64
dtype: object

In [38]:
df = pd.merge(weather_daily, demand_daily, how='inner', on='date')
display(df.head(5))

,date,temperature,demand
0,2017-01-01,8.0,385445
1,2017-01-02,4.4,417277
2,2017-01-03,5.6,434018
3,2017-01-04,8.2,430935
4,2017-01-05,1.0,459619


In [39]:
df['day'] = df['date'].dt.dayofweek
display(df.head(5)) # 0=mon, ..., sun=6

,date,temperature,demand,day
0,2017-01-01,8.0,385445,6
1,2017-01-02,4.4,417277,0
2,2017-01-03,5.6,434018,1
3,2017-01-04,8.2,430935,2
4,2017-01-05,1.0,459619,3


In [40]:
df['month'] = df['date'].dt.month
display(df.head(5)) # 1=jan, ..., dec=12

,date,temperature,demand,day,month
0,2017-01-01,8.0,385445,6,1
1,2017-01-02,4.4,417277,0,1
2,2017-01-03,5.6,434018,1,1
3,2017-01-04,8.2,430935,2,1
4,2017-01-05,1.0,459619,3,1


In [41]:
# add (find) bank holiday (binary mask?)
holidays = calendar().holidays(start=df['date'].min(), end=df['date'].max())
df['holiday'] = df['date'].isin(holidays).astype(int)
display(df.head(5))

,date,temperature,demand,day,month,holiday
0,2017-01-01,8.0,385445,6,1,0
1,2017-01-02,4.4,417277,0,1,1
2,2017-01-03,5.6,434018,1,1,0
3,2017-01-04,8.2,430935,2,1,0
4,2017-01-05,1.0,459619,3,1,0


In [42]:
# Login to hopsworks project
project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5295
Connected. Call `.close()` to terminate connection gracefully.


In [43]:
# Create feature group
fg = fs.get_or_create_feature_group(
    name="ny_elec",
    primary_key=["date", "temperature", "day", "month", "holiday"],
    version=1,
    description="Dataset of NY region electricity demand with average daily weather and bank holidays")


In [44]:
# Insert feature group to hopsworks 
fg.insert(df, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/5295/fs/5215/fg/13782


Uploading Dataframe: 0.00% |          | Rows 0/1826 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5295/jobs/named/ny_elec_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f80d28e4400>, None)